In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get("https://www.shl.com/solutions/products/product-catalog/")
wait = WebDriverWait(driver, 10)

# STEP 1: Collect all assessment links with pagination
assessment_links = set()

while True:
    # wait for the assessment links to load
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/product-catalog/view/']")))
    time.sleep(2)

    # collect current page's assessment links
    for elem in driver.find_elements(By.CSS_SELECTOR, "a[href*='/product-catalog/view/']"):
        link = elem.get_attribute('href')
        if link:
            assessment_links.add(link)

    # find all "Next" buttons on page
    next_buttons = driver.find_elements(By.LINK_TEXT, "Next")

    # if there are at least 2, the second is the Individual Test Solutions pagination
    if len(next_buttons) >= 2:
        btn = next_buttons[1]
    # fallback: if only one Next exists, click it
    elif len(next_buttons) == 1:
        btn = next_buttons[0]
    else:
        print("No more pages to paginate. Finished collecting links.")
        break

    # click and loop
    try:
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(2)
    except Exception:
        print("Failed to click Next. Exiting pagination loop.")
        break

print(f"Total unique assessments found: {len(assessment_links)}")
def normalize_url(url):
    return url.rstrip('/')

driver.get("https://www.shl.com/solutions/products/product-catalog/")
adaptive_map = {}

while True:
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr")))
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        try:
            link_elem = row.find_element(By.CSS_SELECTOR, "a[href*='/product-catalog/view/']")
            link = normalize_url(link_elem.get_attribute('href'))
            adaptive = row.find_elements(By.CSS_SELECTOR, "td")[-2].text.strip()
            adaptive_map[link] = adaptive
        except Exception:
            continue

    next_buttons = driver.find_elements(By.LINK_TEXT, "Next")
    if len(next_buttons) >= 2:
        btn = next_buttons[1]
    elif len(next_buttons) == 1:
        btn = next_buttons[0]
    else:
        break

    try:
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(2)
    except Exception:
        break

print(f"✅ Collected Adaptive/IRT for {len(adaptive_map)} assessments.")


# STEP 2: Visit each assessment page and extract details
data = []

for link in assessment_links:
    driver.get(link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # -------- Extract Title --------
    try:
        title = driver.find_element(By.TAG_NAME, 'h1').text.strip()
    except:
        title = None

    # -------- Extract Metadata --------
    metadata = {}
    for h4 in driver.find_elements(By.CSS_SELECTOR, "h4"):
        key = h4.text.strip().lower().rstrip(':')
        try:
            val = h4.find_element(By.XPATH, "following-sibling::p[1]").text.strip()
        except:
            val = None
        metadata[key] = val

    description = metadata.get('description')
    if not description:
        try:
            description = driver.find_element(By.CSS_SELECTOR, 'div.description').text.strip()
        except:
            description = None

    job_levels = metadata.get('job levels')
    languages = metadata.get('languages') or metadata.get('report language availability')

    raw_length = metadata.get('assessment length', '') or metadata.get('completion time', '')
    match = re.search(r'(\d+)', raw_length)
    length = match.group(1) if match else None

      # -------- ✅ Extract Test Type (Final, working method) --------
    test_types = []
    try:
        p = driver.find_element(
            By.XPATH,
            "//p[starts-with(normalize-space(text()), 'Test Type:')]"
        )
        raw = p.text  # e.g. "Test Type: A B P"
        test_types = re.findall(r'\b[A-Z]\b', raw)
    except:
        test_types = []
        # -------- Extract Remote Testing (by looking for the green dot) --------
    remote_testing = False
    try:
        # look for the <p> whose text starts with "Remote Testing:"
        for p in driver.find_elements(By.CSS_SELECTOR, "p.product-catalogue__small-text"):
            if p.text.strip().startswith("Remote Testing"):
                # if a green circle is present it will have class "catalogue__circle -yes"
                dots = p.find_elements(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
                remote_testing = len(dots) > 0
                break
    except Exception:
        remote_testing = False


    # -------- Append Extracted Data --------
    data.append({
        'Title': title,
        'Description': description,
        'Job Levels': job_levels,
        'Languages': languages,
        'Assessment Length': length,
        'Test Type': ", ".join(test_types),
        'Remote Testing': remote_testing,
        'Adaptive/IRT': adaptive_map.get(normalize_url(link)),  # 🔥 Normalize URL for match

        'URL': link
    })

# STEP 3: Save to CSV
df = pd.DataFrame(data)
df.to_csv('NEW_individual_assess.csv', index=False)
print("✅ Saved to shl_individual_assess.csv")
driver.quit() 

No more pages to paginate. Finished collecting links.
Total unique assessments found: 381
✅ Saved to shl_individual_assess.csv


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get("https://www.shl.com/solutions/products/product-catalog/")
wait = WebDriverWait(driver, 10)

# STEP 1: Collect all assessment links with pagination
assessment_links = set()

while True:
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/product-catalog/view/']")))
    time.sleep(2)
    for elem in driver.find_elements(By.CSS_SELECTOR, "a[href*='/product-catalog/view/']"):
        link = elem.get_attribute('href')
        if link:
            assessment_links.add(link)

    try:
        next_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Next")))
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
    except:
        print("No more pages to paginate. Finished collecting links.")
        break

print(f"Total unique assessments found: {len(assessment_links)}")
# First, go back to the main catalog page and collect Adaptive/IRT values from the table
driver.get("https://www.shl.com/solutions/products/product-catalog/")
time.sleep(2)

adaptive_map = {}
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

for row in rows:
    try:
        link_elem = row.find_element(By.CSS_SELECTOR, "a[href*='/product-catalog/view/']")
        link = link_elem.get_attribute('href')
        adaptive = row.find_elements(By.CSS_SELECTOR, "td")[-2].text.strip()  # 2nd last td is Adaptive/IRT
        adaptive_map[link] = adaptive
    except:
        continue

# STEP 2: Visit each assessment page and extract details
data = []

for link in assessment_links:
    driver.get(link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # -------- Extract Title --------
    try:
        title = driver.find_element(By.TAG_NAME, 'h1').text.strip()
    except:
        title = None

    # -------- Extract Metadata --------
    metadata = {}
    for h4 in driver.find_elements(By.CSS_SELECTOR, "h4"):
        key = h4.text.strip().lower().rstrip(':')
        try:
            val = h4.find_element(By.XPATH, "following-sibling::p[1]").text.strip()
        except:
            val = None
        metadata[key] = val

    description = metadata.get('description')
    if not description:
        try:
            description = driver.find_element(By.CSS_SELECTOR, 'div.description').text.strip()
        except:
            description = None

    job_levels = metadata.get('job levels')
    languages = metadata.get('languages') or metadata.get('report language availability')

    raw_length = metadata.get('assessment length', '') or metadata.get('completion time', '')
    match = re.search(r'(\d+)', raw_length)
    length = match.group(1) if match else None

    # -------- ✅ Extract Test Type (Final, working method) --------
    test_types = []
    try:
        p = driver.find_element(
            By.XPATH,
            "//p[starts-with(normalize-space(text()), 'Test Type:')]"
        )
        raw = p.text  # e.g. "Test Type: A B P"
        test_types = re.findall(r'\b[A-Z]\b', raw)
    except:
        test_types = []

   # -------- Extract Remote Testing (by looking for the green dot) --------
    remote_testing = False
    try:
        # look for the <p> whose text starts with "Remote Testing:"
        for p in driver.find_elements(By.CSS_SELECTOR, "p.product-catalogue__small-text"):
            if p.text.strip().startswith("Remote Testing"):
                # if a green circle is present it will have class "catalogue__circle -yes"
                dots = p.find_elements(By.CSS_SELECTOR, "span.catalogue__circle.-yes")
                remote_testing = len(dots) > 0
                break
    except Exception:
        remote_testing = False


    # --------  Append Extracted Data --------
    data.append({
        'Title': title,
        'Description': description,
        'Job Levels': job_levels,
        'Languages': languages,
        'Assessment Length': length,
        'Test Type': ", ".join(test_types),
        'Remote Testing': remote_testing,
        'Adaptive/IRT': adaptive_map.get(link),  # <-- 🔥 New field here
        'URL': link
    })

# STEP 3: Save to CSV
df = pd.DataFrame(data)
df.to_csv('shl_pre_pack.csv', index=False)
print("✅ Saved to NEW_pre_pack.csv")
driver.quit()

No more pages to paginate. Finished collecting links.
Total unique assessments found: 153
✅ Saved to NEW_pre_pack.csv


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

# 1) Point to Pre‑packaged URL only
URL = "https://www.shl.com/products/product-catalog/?type=0"

# 2) Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get(URL)
time.sleep(4)  # give JS time to render the single table

assessments = []

while True:
    print("📄 Scraping current page…")

    # 3) Extract rows from the Pre‑packaged table
    table = driver.find_element(By.CSS_SELECTOR, "div.custom__table-responsive > table")
    rows  = table.find_elements(By.CSS_SELECTOR, "tbody tr")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) < 4:
            continue

        link = cells[0].find_element(By.TAG_NAME, "a")
        assessments.append({
            "Title":         link.text.strip(),
            "URL":           link.get_attribute("href"),
            "RemoteTesting": bool(cells[1].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes")),
            "Adaptive/IRT":  bool(cells[2].find_elements(By.CSS_SELECTOR, ".catalogue__circle.-yes")),
            "TestTypes":     ", ".join([e.text for e in cells[3].find_elements(By.CSS_SELECTOR, ".product-catalogue__key")])
        })

    # 4) Handle pagination via the global pager
    try:
        next_li = driver.find_element(
            By.CSS_SELECTOR,
            "ul.pagination li.pagination__item.-arrow.-next"
        )
        # If this li has "-disabled", we're done
        if "-disabled" in next_li.get_attribute("class"):
            print("⛔ Reached last page; stopping.")
            break

        # Otherwise click its <a> child to advance
        next_li.find_element(By.TAG_NAME, "a").click()
        time.sleep(3)  # wait for new page to load

    except NoSuchElementException:
        # Pager is missing or changed – exit gracefully
        print("⚠️ Cannot find next arrow; exiting pagination loop.")
        break
    
driver.quit()

# 5) Save to CSV
df = pd.DataFrame(assessments)
df.to_csv("shl_prepackaged_only.csv", index=False)
print(f"✅ Extracted {len(df)} rows → shl_prepackaged_only.csv")


📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
📄 Scraping current page…
⛔ Reached last page; stopping.
✅ Extracted 141 rows → shl_prepackaged_only.csv


In [1]:
import pandas as pd

# Load the two CSV files
df1 = pd.read_csv('C:\\Users\\vidhi\\Downloads\\my_shl\\shl_prepackaged_no_duplicates_table.csv')
df2 = pd.read_csv('C:\\Users\\vidhi\\Downloads\\my_shl\\indiv_table_no_duplicates.csv')

# Rename columns in df2 to match df1 (handle spacing and casing differences)
#df2.columns = ['Title', 'URL', 'RemoteTesting', 'Adaptive/IRT', 'Test Types']

# Concatenate both DataFrames row-wise
merged_df = pd.concat([df1, df2], ignore_index=True)

# Optional: Remove duplicate rows (if needed)
merged_df = merged_df.drop_duplicates()

# Save to new CSV
merged_df.to_csv('merged_table_output.csv', index=False)

print("CSV files merged successfully and saved to 'merged_output.csv'")


CSV files merged successfully and saved to 'merged_output.csv'


In [2]:
import pandas as pd

# Load the two CSV files
df1 = pd.read_csv('C:\\Users\\vidhi\\Downloads\\my_shl\\shl_pre_pack.csv')
df2 = pd.read_csv('C:\\Users\\vidhi\\Downloads\\my_shl\\shl_individual_assess.csv')


# Concatenate both DataFrames row-wise
merged_df = pd.concat([df1, df2], ignore_index=True)

# Optional: Remove duplicate rows (if needed)
merged_df = merged_df.drop_duplicates()

# Save to new CSV
merged_df.to_csv('merged_detail_output.csv', index=False)

print("CSV files merged successfully and saved to 'merged_output.csv'")


CSV files merged successfully and saved to 'merged_output.csv'


In [3]:
import pandas as pd

# Load the first CSV
df1 = pd.read_csv('C:\\Users\\vidhi\\Downloads\\my_shl\\merged_detail_output.csv')

# Load the second CSV (only URL and Adaptive/IRT columns)
df2 = pd.read_csv('C:\\Users\\vidhi\\Downloads\\my_shl\\merged_table_output.csv')[['URL', 'Adaptive/IRT']]

# Merge Adaptive/IRT into df1 based on URL
merged_df = pd.merge(df1, df2, on='URL', how='left')

# Save to a new CSV
merged_df.to_csv('final.csv', index=False)

print("Merged file saved as 'first_file_with_adaptive_irt.csv'")


Merged file saved as 'first_file_with_adaptive_irt.csv'


In [4]:
import pandas as pd

# Load CSV file
df = pd.read_csv("C:\\Users\\vidhi\\Downloads\\my_shl\\final.csv")

# Find duplicate rows (excluding the index)
duplicates = df[df.duplicated()]

# Print duplicates
print("Duplicate rows:")
print(duplicates)




Duplicate rows:
Empty DataFrame
Columns: [Title, Description, Job Levels, Languages, Assessment Length, TestTypes, RemoteTesting, URL, Adaptive/IRT]
Index: []


In [1]:
import pandas as pd
import re

# Load raw CSV
df = pd.read_csv("final.csv")

# Rename columns to consistent lowercase, underscore-separated format
df = df.rename(columns={
    "Title": "name",
    "Description": "description",
    "Job Levels": "job_levels",
    "Languages": "languages",
    "Assessment Length": "duration_minutes",
    "TestTypes": "test_type",
    "RemoteTesting": "is_remote",
    "URL": "url",
    "Adaptive/IRT": "is_adaptive"
})

# Strip whitespace and handle missing values
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df["job_levels"] = df["job_levels"].fillna("").str.split(",")
df["languages"] = df["languages"].fillna("").str.split(",")

# Expand test_type abbreviations
type_map = {
    "A": "Ability",
    "B": "Behavioral",
    "C": "Cognitive",
    "P": "Personality",
    "S": "Skills"
}
def map_test_types(raw):
    if pd.isna(raw): return []
    types = [x.strip() for x in raw.split(",")]
    return [type_map.get(t, t) for t in types]

df["test_type"] = df["test_type"].apply(map_test_types)

# Convert types
df["duration_minutes"] = pd.to_numeric(df["duration_minutes"], errors="coerce").fillna(0).astype(int)
df["is_remote"] = df["is_remote"].astype(bool)
df["is_adaptive"] = df["is_adaptive"].astype(bool)

# Normalize and truncate descriptions
def normalize_description(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s.,!?;:()/-]", "", text)
    return ' '.join(text.split()[:300])  # limit to first 300 words

df["description"] = df["description"].apply(normalize_description)

# Drop rows missing key fields
df = df.dropna(subset=["name", "description", "url"])
df = df.drop_duplicates(subset=["name"])

# Save cleaned version
df.to_csv("shl_assessments_cleaned.csv", index=False)
print("✅ Cleaned data saved to shl_assessments_cleaned.csv")


✅ Cleaned data saved to shl_assessments_cleaned.csv


C:\Users\vidhi\AppData\Local\Temp\ipykernel_19684\1332143791.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [6]:
import pandas as pd
import ast

# Load the dataset
df = pd.read_csv("shl_assessments_cleaned.csv")

# Helper function to clean list-like string columns
def clean_list_column(series):
    def safe_parse(val):
        try:
            parsed = ast.literal_eval(val)
            return [x.strip().lower() for x in parsed if isinstance(x, str) and x.strip()]
        except Exception:
            return []
    return series.apply(safe_parse)

# Clean list-type columns
list_columns = ['job_levels', 'languages', 'test_type']
for col in list_columns:
    df[col] = clean_list_column(df[col])

# Normalize boolean columns
df['remote_support'] = df['is_remote'].astype(str).str.upper().map({'TRUE': 'Yes', 'FALSE': 'No'})
df['adaptive_support'] = df['is_adaptive'].astype(str).str.upper().map({'TRUE': 'Yes', 'FALSE': 'No'})

# Ensure duration is numeric
df['duration_minutes'] = pd.to_numeric(df['duration_minutes'], errors='coerce')

# Optionally drop or warn about rows with NaN durations
df = df.dropna(subset=['duration_minutes'])

# Final cleaned dataframe
cleaned_df = df.drop(columns=['is_remote', 'is_adaptive'])

cleaned_df.to_csv("cleaned_dataset.csv", index=False)

print("✅ Dataset cleaned successfully.")


✅ Dataset cleaned successfully.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Navigate to the SHL product catalog page
driver.get("https://www.shl.com/products/product-catalog/?start=0&type=1&type=1")
time.sleep(5)  # Wait for the page to load completely

# Initialize a list to store the extracted data
assessments = []

while True:
    print("📄 Scraping current page...")

    # Locate all table rows in the catalog
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

    for row in rows:
        try:
            # Extract all table data cells in the row
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 4:
                continue  # Skip rows that don't have enough columns

            # Extract assessment title and URL
            title_element = cells[0].find_element(By.TAG_NAME, "a")
            title = title_element.text.strip()
            url = title_element.get_attribute("href")

            # Determine if Remote Testing is available
            remote_testing = bool(cells[1].find_elements(By.CLASS_NAME, "catalogue__circle"))

            # Determine if Adaptive/IRT is available
            adaptive_irt = bool(cells[2].find_elements(By.CLASS_NAME, "catalogue__circle"))

            # Extract Test Types
            test_type_elements = cells[3].find_elements(By.CLASS_NAME, "product-catalogue__key")
            test_types = [elem.text.strip() for elem in test_type_elements]

            # Append the extracted information to the list
            assessments.append({
                "Title": title,
                "URL": url,
                "Remote Testing": remote_testing,
                "Adaptive/IRT": adaptive_irt,
                "Test Types": ", ".join(test_types)
            })
        except Exception as e:
            print(f"Error processing row: {e}")
            continue

    # Try to find and click the "Next" button
    try:
        next_buttons = driver.find_elements(By.LINK_TEXT, "Next")
        if not next_buttons:
            print("⛔ No more pages to paginate. Exiting.")
            break

        next_button = next_buttons[-1]  # Use the last Next button
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Wait for the next page to load
    except Exception as e:
        print(f"⚠️ Failed to click Next button: {e}")
        break

# Close the WebDriver
driver.quit()

# Convert the list to a DataFrame and save to CSV
df = pd.DataFrame(assessments)
df.to_csv("shl_pre_pack_table.csv", index=False)
print("✅ Data extraction complete. Saved to 'shl_assessments_with_pagination.csv'.")


⚠️ Couldn't ensure Individual Test Solutions tab is selected: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(text(), 'Individual Test Solutions')]"}
  (Session info: chrome=135.0.7049.116); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0053D363+60275]
	GetHandleVerifier [0x0053D3A4+60340]
	(No symbol) [0x003706F3]
	(No symbol) [0x003B8690]
	(No symbol) [0x003B8A2B]
	(No symbol) [0x00400EE2]
	(No symbol) [0x003DD0D4]
	(No symbol) [0x003FE6EB]
	(No symbol) [0x003DCE86]
	(No symbol) [0x003AC623]
	(No symbol) [0x003AD474]
	GetHandleVerifier [0x00788FE3+2467827]
	GetHandleVerifier [0x007845E6+2448886]
	GetHandleVerifier [0x0079F80C+2560028]
	GetHandleVerifier [0x00553DF5+153093]
	GetHandleVerifier [0x0055A3BD+179149]
	GetHandleVerifier [0x00544BB8+91080]
	GetHandleVerifier [0x00544D60+91504]
	GetHandleVerifier [0

KeyboardInterrupt: 